DEPENDENCIES

In [1]:

import sys
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm
from scipy.optimize import fmin_slsqp as slsqp
from datetime import datetime
from math import exp, isnan
try:
    import cvxpy as cp    
except:
    print ('This version of higher_moments requires installation of cvxpy.')
    raise

GET ASSET GROUP CATEGORIES

In [2]:
# get asset group codes
def load_asset_codes(codefile=None):
    try:
        equivs=pd.read_csv(codefile).values
        equivs=[x if isinstance(x[1],str) else [x[0],'NOGROUP'] for x in equivs]
        lookup_group={}
        for tick in equivs:
            lookup_group[tick[0]]=tick[1]
        members={}
        categories=sorted(set([x[1] for x in equivs]))
        for group in categories:
            members[group]=[x[0] for x in equivs if x[1]==group]     
        return(lookup_group,members)
    except:
        print('NO ASSET CATEGORIZATION FOUND')
        raise

#a,b=load_asset_codes('CDATA40/equiv.csv')
#print(b)

LOAD INPUT PRICE FILE

In [3]:
def load_source(sourcefile):
    try:
        source=pd.read_csv(sourcefile)
        temp=source.get('Date')
        if not temp is None:
            source.index=temp 
            source=source.drop(columns=['Date'])
        return source
    except:
        print('NO SOURCE FOUND')
        return None

REARRANGE COLUMNS BY ASSET CLASS IF SOURCE IS RETURN FILE

In [4]:
def rearrange(members,source):
    cols=[tick for group in members for tick in members[group]]
    source2=pd.DataFrame(columns=cols,index=source.index)
    for tick in cols:
        source2[tick]=source[tick]
    return(source2)

CALCULATE RETURNS

In [5]:
def calculate_returns(prices2,return_interval):
    price_data=np.array(prices2.values,dtype='float32')
    price_data1=np.ones((price_data.shape[0],price_data.shape[1]))
    price_data1[return_interval:]=price_data[:-return_interval]
    returns=(price_data/price_data1)
    returns=returns[return_interval:]-1. 
    returns_df=pd.DataFrame(returns)   
    returns_df.columns=prices2.columns
    returns_df.index=prices2.index[return_interval:]
    return(returns_df)

COLLECT FURTHER STATISTICS

In [6]:

def rtn_summary(returns,tickers):
    means=np.mean(returns,axis=0) 
    covs=np.cov(returns.T)
    stdevs=np.std(returns,axis=0)
    corrs=np.round(np.corrcoef(returns.T),2)
    corr=pd.DataFrame(corrs,columns=tickers)
    means=pd.Series(means)
    stdev=pd.Series(stdevs)
    skews=stats.skew(returns,axis=0)
    skew=pd.Series(skews)
    kurts=stats.kurtosis(returns,axis=0,fisher=False)
    kurt=pd.Series(kurts)
    descript=pd.DataFrame({'TICKER':tickers,'MEAN':np.round(means,4),
        'STDEV':np.round(stdev,4),'SKEW':np.round(skew,2),'KURT':np.round(kurt,2)})
    combined=descript.join(corr,how='outer')      
        
    return (means,covs,skews,kurts,combined)
#means,covs,skews,kurts=rtn_summary(returns)

SLSQP HILL-CLIMBING OPTIMIZER

In [7]:
def default_init_allocate(returns_df):
    ncols=len(returns_df.columns)
    def_init_alloc=np.zeros(ncols,dtype='float64')
    stdevs=np.std(returns_df,axis=0)
    mn=np.min(stdevs)
    idx=[i for i,v in enumerate(stdevs) if v==mn][0]
    def_init_alloc[idx]=1.0        
    return def_init_alloc

def eqcons1(x,*args): #budget constraint
    return(np.array([np.sum(x)-1.0]))

def mobjective(x,means,covariance,risk_aversion):
    mn=np.dot(x,means)
    cv=np.matmul(x.T,covariance)
    cv2=np.dot(cv,x)
    return(-mn+risk_aversion*cv2/2.0)

def wobjective2(x,levreturns,worst,leverage):
    levportreturn=np.matmul(levreturns,x)
    squeeze=(1.0+worst)/leverage  #for compressing over 100% losses to nearly 100%    
    new_worst=worst+(levportreturn+1.0)*squeeze #compressing
    safeportreturn=np.maximum(levportreturn,new_worst)     
    log_portreturn=np.log1p(safeportreturn) #log(1+X) numpy function
    return (-np.sum(log_portreturn))

#-----------------------------------------------------------

def hill_climb(returns,levs,means,covs,headers,labels,long_only,worst,ob,alloc_mat=None):
    merit=pd.DataFrame(index=headers,columns=labels)
    ncols2=returns.shape[1]
    nrows2=returns.shape[0]
    nlevs=len(levs)   
    alloc=np.ones((nlevs,ncols2),dtype='float64')
    prtns=np.zeros((nlevs,nrows2),dtype='float64')
    returns_df=pd.DataFrame(returns)
    def_init_alloc=default_init_allocate(returns_df)
    if not long_only:
        print('Not set up for long-short problems')
        raise
    #long-only bounds
    upper=np.ones(ncols2,dtype='float64')
    lower=np.zeros(ncols2,dtype='float64')
    bounds=list(zip(lower,upper))
   
    for i in range(nlevs):     #in this use where called by cvxpy call, keep iteration over single leverage  
        lev=levs[i]
        levreturn=returns*lev
        
        if alloc_mat is None:
            x0=default_init_allocate(returns_df)
        else:
            x0=alloc_mat[i]
            
        if ob=='MV':
            # run mean_variance optimizer                                   
            temp=slsqp(mobjective,x0,eqcons=[eqcons1],args=tuple((means,covs,lev)),bounds=bounds,full_output=True,acc=1e-07) 
            alloc[i]=temp[0]
        
        if ob=='LLS':
            # run LLS optimizer
            levreturns=returns*lev      
            temp=slsqp(wobjective2,x0,eqcons=[eqcons1],args=tuple((levreturns,worst,lev)),bounds=bounds,full_output=True,acc=1e-08)
            alloc[i]=temp[0]
    return (None,alloc[::],None)


CVXPY OPTIMIZER

In [8]:
def optim_with_cvxpy2(orig_rtns,mrtns,levs,mmeans,mcovs,headers,labels,long_only,worst,ob):
    barrier=worst+1
    merit=pd.DataFrame(index=headers,columns=labels)
    nrows,ncols=mrtns.shape
    nlevs=len(levs)
    mns=mmeans.values
    orig_means=np.mean(orig_rtns,axis=0)
    orig_covs=np.cov(orig_rtns.T)
    alloc=np.ones((nlevs,ncols),dtype='float64')
    prtns=np.zeros((nlevs,nrows),dtype='float64')
    
    xx=cp.Variable(ncols)
    for i in range(nlevs):
        lev=levs[i]
        mlevreturn=(mrtns*lev)
        orig_levreturn=(orig_rtns*lev)
        #print(' ')       
        print("Risk Aversion: ",lev)
        if ob=='MV':
            if long_only:
                constraints =[sum(xx)==1, 0<=xx, xx<=1] #Long-only portfolios
            else:
                constraints = [sum(xx)==1.0] 
            objective=cp.Minimize(-cp.sum(cp.multiply(mns,xx)) + lev*cp.quad_form(xx,mcovs)/2.0)
            prob=cp.Problem(objective,constraints)
            result=prob.solve(solver=cp.CLARABEL,tol_feas=1e-8,tol_gap_abs=1e-8, tol_gap_rel=1e-8, tol_ktratio=1e-8, verbose=False)
            xxvalue=xx.value
            prtns[i]=np.dot(orig_rtns,xxvalue)
            
            merit.loc[headers[i],'M_objective'] = np.sum(orig_means*xxvalue) - levs[i]*np.dot(np.dot(xxvalue,orig_covs),xxvalue.T)/2.0
            merit.loc[headers[i],'W_objective'] = (np.sum(np.log1p(lev*np.dot(orig_rtns,xxvalue.T)))/nrows)/lev
            
        elif ob=='LLS':
            if long_only:
                constraints =[sum(xx)==1, 0<=xx, xx<=1, -1.0+barrier <= mlevreturn @ xx ] #Long-only portfolios
            else:
                constraints = [sum(xx)==1,-1.0+barrier <= mlevreturn @ xx ]
            objective=cp.Minimize(cp.sum(-cp.log1p(mlevreturn@xx)))
            prob=cp.Problem(objective,constraints)
            result=prob.solve(solver=cp.CLARABEL,tol_feas=1e-8,tol_gap_abs=1e-8, tol_gap_rel=1e-8, tol_ktratio=1e-8, verbose=False) /nrows
            xxvalue=xx.value
            if xxvalue is None:                
                print('WARNING!!!! cvxpy problem may not be feasible.')
                print('Using sqslp with catastrophic returns converted to less extreme losses.')
                dummy1,wallocz,dummy2=hill_climb(mrtns,[lev],mmeans,mcovs,headers,labels,long_only,worst,ob,alloc_mat=None)
                xxvalue=wallocz[0]
                
            prtns[i]=np.dot(orig_rtns,xxvalue)
            merit.loc[headers[i],'M_objective']= sum(orig_means*xxvalue) - levs[i]*np.dot(np.dot(xxvalue,orig_covs),xxvalue.T)/2.0
            merit.loc[headers[i],'W_objective']= (np.sum(np.log1p(np.dot(orig_levreturn,xxvalue)))/nrows)/lev        
        alloc[i]=xxvalue        
    
    return (prtns[::].T,alloc[::],pd.DataFrame.copy(merit,deep=True))


CALL ALTERNATIVE ALLOCATION OPTIMIZERS AND DESCRIBE OBJECTIVE RESULTS AND DIVERSIFICATION IN-SAMPLE

In [9]:
def find_best_allocation(rtns_df,mrtns_df,long_only,worst,levs,lookup_group=None):
    rtns=rtns_df.values
    mrtns=mrtns_df.values
    rtns_cols=rtns_df.columns
    means=np.mean(rtns_df,axis=0)
    mmeans=np.mean(mrtns_df,axis=0)
    covs=np.cov(rtns_df.T)
    mcovs=np.cov(mrtns_df.T)
    headers=['L:'+x for x in list(map(str,levs))]
    labels=['W_objective','M_objective']
    
    print('RUNNING MEAN-VARIANCE OPTIMIZATION')
    mpreturns,malloc,M_merit=optim_with_cvxpy2(rtns,mrtns,levs,mmeans,mcovs,headers,labels,long_only,worst,ob="MV")
    print(' ')
    print('RUNNING EXPECTED SURPLUS GROWTH OPTIMIZATION')
    wpreturns,walloc,W_merit=optim_with_cvxpy2(rtns,mrtns,levs,mmeans,mcovs,headers,labels,long_only,worst,ob="LLS")
    
    print(' ')
    print('ALLOCATIONS TO MAXIMIZE MEAN-VARIANCE')  
    print(pd.DataFrame(np.round(malloc,5),columns=rtns_df.columns,index=headers).T)
    print(' ')
    print('ALLOCATIONS TO MAXIMIZE EXPECTED SURPLUS GROWTH')
    print(pd.DataFrame(np.round(walloc,5),columns=rtns_df.columns,index=headers).T)
    print(' ')
    print('INCREMENTAL ALLOCATIONS')
    dalloc=np.subtract(walloc,malloc)
    print(pd.DataFrame(np.round(dalloc,4),columns=rtns_df.columns,index=headers).T)
    print(' ')
    print('IN-SAMPLE ALLOCATION MERIT FROM MEAN-VARIANCE')
    print(M_merit[['W_objective','M_objective']].head(10))
    print(' ')    
    print('IN-SAMPLE ALLOCATION MERIT FROM EXPECTED SURPLUS GROWTH')
    print(W_merit[['W_objective','M_objective']].head(10))
    print(' ')
    print('INCREMENTAL MERIT')
    delta_merit=np.subtract(np.array(W_merit),np.array(M_merit))
    D_merit=pd.DataFrame(delta_merit,columns=W_merit.columns, index=W_merit.index)
    #following necessary because np.round fails on NaN
    with pd.option_context('display.float_format', '{:,.5f}'.format):
        print(D_merit)
    
    return (wpreturns,mpreturns,walloc,malloc,W_merit,M_merit)


EXTEND ALLOCATION CONSEQUENCES TO PORTFOLIO RETURN CHARACTERISTICS IN SAMPLE

In [10]:
def show_xray(merit,objective,levs,pmean,pstd,pskew,pkurt):
    #X-ray on optimal in-sample surplus log growth rate objective
    exp_utility=[x for x in merit[objective].values]
    xray=pd.DataFrame([levs,exp_utility,pmean,pstd,pskew,pkurt]).T
    xray.columns=['Leverage','Exp_Log_Gr','mean','stdev','skewness','kurtosis']
    
    xray['Q'] = xray['Leverage']*xray['stdev']/(1+xray['Leverage']*xray['mean'])
    print(' ')
    headers=['L:'+x for x in list(map(str,levs))]
    Q_df=pd.DataFrame([headers,xray['Q']]).T
    Q_df.columns=['Leverage','Q']
    with pd.option_context('display.float_format', '{:,.3f}'.format):
        print(Q_df.to_string(index=False))
    
    xray['First']= np.log1p(xray['Leverage']*xray['mean'])
    xray['Second']=-(xray['Q']**2)/2
    xray['Third']=xray['skewness']*(xray['Q']**3)/3
    xray['Fourth']=-xray['kurtosis']*(xray['Q']**4)/4
    xray['Residual']=xray['Exp_Log_Gr']-xray['First']-xray['Second']-xray['Third']-xray['Fourth']
    xray=xray.drop(['mean','stdev','skewness','kurtosis','Q'],axis=1)
    print(' ')
    
    print('COMPOSITION BY RETURN DISTRIBUTION MOMENTS:')
    print(' ')
    print(xray.to_string(index=False))    
    
    return None

In [11]:
def describe_portfolio_returns(wprtns,mprtns,wmerit,mmerit,levs):
    #COMPARE OUTPUTS ON TRADITIONAL STATISTICS
    print(' ')
    
    wpmean=pd.Series(np.mean(wprtns,axis=0))
    wpstd=pd.Series(np.std(wprtns,axis=0))
    wpskew=pd.Series(stats.skew(wprtns,axis=0))
    wpkurt=pd.Series(stats.kurtosis(wprtns,axis=0,fisher=False))    

    mpmean=pd.Series(np.mean(mprtns,axis=0))
    mpstd=pd.Series(np.std(mprtns,axis=0))
    mpskew=pd.Series(stats.skew(mprtns,axis=0))
    mpkurt=pd.Series(stats.kurtosis(mprtns,axis=0,fisher=False))
    
    pdescribe1=pd.DataFrame({'WMEAN': np.round(wpmean,4),'MMEAN':np.round(mpmean,4)})
    pdescribe2=pd.DataFrame({'WSTD': np.round(wpstd,4),'MSTD':np.round(mpstd,4)})
    pdescribe3=pd.DataFrame({'WSKEW': np.round(wpskew,3),'MSKEW':np.round(mpskew,3)})
    pdescribe4=pd.DataFrame({'WKURT': np.round(wpkurt,3),'MKURT':np.round(mpkurt,3)})   
    pdescribe=pd.concat([pdescribe1,pdescribe2,pdescribe3,pdescribe4],axis=1,sort=False)
    pdescribe.index=['L:'+x for x in list(map(str,levs))]
    print('COMPARE PORTFOLIO STATISTICS')

    print(' ')
    print('IN_SAMPLE SURPLUS GROWTH OBJECTIVE WITH MEAN-VARIANCE OPTIMIZATION')
    show_xray(mmerit,'W_objective',levs,mpmean,mpstd,mpskew,mpkurt)
    print(' ')
    print('IN-SAMPLE SURPLUS GROWTH OBJECTIVE WITH SURPLUS GROWTH OPTIMIZATION')
    show_xray(wmerit,'W_objective',levs,wpmean,wpstd,wpskew,wpkurt)
    print(' ')

    return


RESEARCH RECORDKEEPING

In [12]:
def print_parameters(journalfile,logfile,sample,codefile,sourcefile,sourcetype,Llist,long_only,return_interval,worst):
    print(' ')    
    print(f'{journalfile=}')
    print(f'{logfile=}')
    print(f'{sample=}')
    print(f'{codefile=}')
    print(f'{sourcefile=}')
    print(f'{sourcetype=}')
    print(f'{Llist=}')
    print(f'{long_only=}') 
    print(f'{return_interval=}')
    print(f'{worst=}')
    print(' ')
    return
    
    
    

MAIN PROGRAM

In [13]:
def higher_moments(params={}):

    journalfile=params.get('journalfile')
    logfile=params.get('logfile')
    sample=params.get('sample')
    codefile=params.get('codefile')
    sourcefile=params.get('sourcefile')
    sourcetype=params.get('sourcetype')    
    Llist=params.get('Llist')
    long_only=params.get('long_only')
    return_interval=params.get('return_interval')
    worst=params.get('worst')
   
    #See import dependencies in first cell

    #record run description in journalfile
    orig_stdout = sys.stdout
    e=open(journalfile, 'a')
    sys.stdout=e
    print_parameters(journalfile,logfile,sample,codefile,sourcefile,sourcetype,Llist,long_only,return_interval,worst)
    e.close()

    #record results in logfile
    f = open(logfile, 'w')
    sys.stdout = f

    #record control parameters
    print_parameters(journalfile,logfile,sample,codefile,sourcefile,sourcetype,Llist,long_only,return_interval,worst)

    #Read in asset categories and members
    try:
        lookup_group,members=load_asset_codes(codefile)
        print(' ')
        print('Asset groups and members:')
        print(members)
        print(' ')
    except:
        print('Error: Failed to load codefile')
        raise
        
    #Read in Prices or Returns, based on sourcetype, adjusted for dividends and interest if possible
    
    if sourcetype=='PRICES':
        #prices=load_source(sourcefile) 
        #Rearrange prices by asset class for greater interpretability
        prices2=rearrange(members,load_source(sourcefile) )
        #Calculate return matrix
        returns_df=calculate_returns(prices2,return_interval)
    elif sourcetype=='RETURNS':
        returns_df=rearrange(members,load_source(sourcefile))
    else:
        print('UNABLE TO DETERMINE SOURCE TYPE')
        raise
        
    # optionally...
    print(' ')
    print('RETURNS SAMPLE')
    print(returns_df.head())
    print(' ')
    print(returns_df.tail())
    print(' ')

    print('RETURNS DESCRIPTION')
    print(returns_df.describe())
    
    # Do mean-variance and log leveraged surplus optimizations
    (wpreturns,mpreturns,walloc,malloc,
        W_merit,M_merit) = find_best_allocation(returns_df,
        returns_df,long_only,worst,Llist,lookup_group)

    #describe portfolio return statistics for different leverages
    describe_portfolio_returns(wpreturns,mpreturns,W_merit, M_merit,Llist)

    #close logfile and print it on terminal
    f.close()
    sys.stdout = orig_stdout

    h=open(logfile,'r')
    for line in h:
        if len(line)>0:          
            print(line[:-1])          
    h.close()
        
    optimizer_output={
        "params":params,
        "members":members,
        "returns_df":returns_df,
        "wpreturns":wpreturns,
        "mpreturns":mpreturns,
        "walloc":walloc,
        "malloc":malloc,
        "W_merit":W_merit,
        "M_merit": M_merit,
    }

    
    print(' ')
    print('DONE!')
    
    return optimizer_output

SET PARAMETERS AND CALL RESEARCH OPTIMIZER

In [14]:
#set parameters

params=dict(
    journalfile='JOURNAL.txt',
    logfile='RUNxx.txt',
    sample='20YR',
    codefile='CDATA20/equiv.csv',
    sourcefile='CDATA20/prices.csv',
    sourcetype='PRICES',
    Llist=[1,2],
    long_only=True,
    return_interval=1,
    worst=(-0.99),
    )
#run main program
optimizer_output=higher_moments(params)


 
journalfile='JOURNAL.txt'
logfile='RUNxx.txt'
sample='20YR'
codefile='CDATA20/equiv.csv'
sourcefile='CDATA20/prices.csv'
sourcetype='PRICES'
Llist=[1, 2]
long_only=True
return_interval=1
worst=-0.99
 
 
Asset groups and members:
{'BOND': ['VFIIX', 'VUSTX', 'VWESX', 'VFITX'], 'CASH': ['VWSTX'], 'JUNK': ['VWEHX', 'VWAHX'], 'STOCK': ['XLE', 'EWJ', 'XLP', 'XLV', 'XLY', 'DIA', 'XLK', 'SPY']}
 
 
RETURNS SAMPLE
               VFIIX     VUSTX     VWESX     VFITX     VWSTX     VWEHX  \
Date                                                                     
1999-01-01  0.008281  0.009904  0.028578  0.006580  0.004880  0.019328   
1999-02-01 -0.007106 -0.047315 -0.036633 -0.030332  0.001924 -0.005854   
1999-03-01  0.006325 -0.015088 -0.005120  0.005414  0.000833  0.007603   
1999-04-01  0.004371  0.013424  0.004758  0.003062  0.002508  0.013409   
1999-05-01 -0.008280 -0.017152 -0.016449 -0.013952  0.001801 -0.016409   

               VWAHX       XLE       EWJ       XLP       XLV       XLY

/Users/jarrodwilcox/pydev13/venv/lib/python3.13/site-packages/cvxpy/problems/problem.py:1510: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
